<a href="https://colab.research.google.com/github/mjgroth/deephys-aio/blob/master/Python_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Deephys



In [1]:
#@title 1. Install Deephys From PIP

!pip install deephys==0.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 KB 14.2 MB/s eta 0:00:00


In [2]:
#@title 2. Import Names From Deephys

from deephys import *

# Run Neural Network Experiment

In [3]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Import Libraries
from __future__ import print_function
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

#Argument initializatio
args={}
kwargs={}
args['batch_size']=128
args['cuda']=True #Enabling cuda is required

#load the data (with normalization)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=args['batch_size'],
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=args['batch_size'],
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
#Neural network model definition (Different ResNet versions)
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear1 = nn.Linear(512*block.expansion, 50)
        self.linear2 = nn.Linear(50, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        out = self.linear2(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


# def ResNet34():
#     return ResNet(BasicBlock, [3, 4, 6, 3])


# def ResNet50():
#     return ResNet(Bottleneck, [3, 4, 6, 3])


# def ResNet101():
#     return ResNet(Bottleneck, [3, 4, 23, 3])


# def ResNet152():
#     return ResNet(Bottleneck, [3, 8, 36, 3])


# def test():
#     net = ResNet18()
#     y = net(torch.randn(1, 3, 32, 32))
#     print(y.size())

Please download [this Google Drive file](https://drive.google.com/file/d/1oUMHbE9ck1Wgi_C-RwmnP_w64IFdL3uK/view?usp=sharing) (`cifar_resnet18.pth`) and [this one](https://drive.google.com/file/d/1VtgCfn5YqYs1zJPyj6b5suv_xHlpJ_K1/view?usp=sharing) (`cifar102_train.npz`) too to continue

Or use PyDrive below (tutorial reference [link text](https://sigmundojr.medium.com/how-do-i-read-a-csv-file-from-google-drive-using-python-colab-966091922852))

In [6]:
#@title Install PyDrive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials

In [7]:
#@title Authenticate PyDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
#@title Get File 1

fileDownloaded = drive.CreateFile({"id":"1oUMHbE9ck1Wgi_C-RwmnP_w64IFdL3uK"})
fileDownloaded.GetContentFile("cifar_resnet18.pth")

In [9]:
#@title Get File 2

fileDownloaded = drive.CreateFile({"id":"1VtgCfn5YqYs1zJPyj6b5suv_xHlpJ_K1"})
fileDownloaded.GetContentFile("cifar102_train.npz")

In [10]:
#Select a model
model = ResNet18()
if args['cuda']:
    model.cuda()

#Load the saved model
state_dict = torch.load('cifar_resnet18.pth')
model.load_state_dict(state_dict)

#This is for CIFAR10
#Save the activations for penultimate layer for further experiments
from torch.utils.data import dataset
from collections import defaultdict
import numpy as np
all_activs = []
all_outputs = []
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

a_s = defaultdict(list)
iterator = iter(testloader)

for _ in range(len(testloader)):
  data, target = next(iterator)
  if args['cuda']:
    data, target = data.cuda(), target.cuda()

  model.linear1.register_forward_hook(get_activation('linear1'))
  # all_outputs.append(model(data))
  activation['output'] = model(data).detach().cpu()#.numpy()
  all_outputs.append(activation['output'])
  activation['linear1'] = F.relu(activation['linear1']).detach().cpu()#.numpy()
  all_activs.append(activation['linear1'])

all_activs = torch.cat(all_activs)
all_outputs = torch.cat(all_outputs)

In [11]:
#Collect neuron activations of penultimate layer for CIFAR10.2 for further experiments
from torch.utils.data import TensorDataset
def get_activation(name):
      def hook(model, input, output):
        activation[name] = output.detach()
      return hook

test_data = np.load('cifar102_train.npz')
all_activs_2 = []
all_outputs_2 = []

test_data_2 = np.transpose(test_data['images'], (0, 3, 1, 2))/255. #Required before the images can be fed to the model
test_data_2[:,0] = (test_data_2[:,0] - 0.4914)/0.2023
test_data_2[:,1] = (test_data_2[:,1] - 0.4822)/0.1994
test_data_2[:,2] = (test_data_2[:,2] - 0.4465)/0.2010
test_data_2 = TensorDataset(torch.FloatTensor(test_data_2), torch.LongTensor(test_data['labels']))
testloader_2 = torch.utils.data.DataLoader(test_data_2,
    batch_size=args['batch_size'], shuffle=False, **kwargs)

activation = {}

a_s_2 = defaultdict(list)
iterator = iter(testloader_2)

for _ in range(len(testloader_2)):
  data, target = next(iterator)
  if args['cuda']:
    data, target = data.cuda(), target.cuda()

  model.linear1.register_forward_hook(get_activation('linear1'))
  activation['output'] = model(data).detach().cpu()#.numpy()
  all_outputs_2.append(activation['output'])
  activation['linear1'] = F.relu(activation['linear1']).detach().cpu()#.numpy()
  all_activs_2.append(activation['linear1'])

all_activs_2 = torch.cat(all_activs_2)
all_outputs_2 = torch.cat(all_outputs_2)

# Save Data With Deephys

In [12]:
#@title Save the model file
neuronList = []
for i in range(np.shape(all_activs)[1]):
  neuronList.append(Neuron())

layerList = []
layerList.append(Layer(
    layerID = "linear1",
    neurons = neuronList
))

#####
neuronList = []
for i in range(np.shape(all_outputs)[1]):
  neuronList.append(Neuron())

layerList.append(Layer(
    layerID = "classification",
    neurons = neuronList
))
#####

model = Model(
    name = "resnet18_cifar",
    suffix = None,
    layers = layerList
)

model.save()

In [13]:
#@title This is for CIFAR10
#Loading the dataset again without normalization
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                   download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=args['batch_size'],
                                        shuffle=False, num_workers=2)


test = import_torch_dataset(
    "CIFARV1",
    testloader.dataset,
    classes,
    [all_activs,all_outputs],
    model
    )
test.suffix = None
for im in test.images:
  im.features = {
      "texture": "elephant"
  }
test.save()

Files already downloaded and verified


In [14]:
#@title This is for CIFAR10.2
#Loading the dataset again without normalization

test_data_2 = np.transpose(test_data['images'], (0, 3, 1, 2))/255.
test_data_2 = TensorDataset(torch.FloatTensor(test_data_2), torch.LongTensor(test_data['labels']))

testloader = torch.utils.data.DataLoader(test_data_2,
    batch_size=args['batch_size'], shuffle=False, **kwargs)

testV2 = import_torch_dataset(
    "CIFARV2",
    testloader.dataset,
    classes,
    [all_activs_2,all_outputs_2],
    model
    )
test.suffix = None
testV2.save()